In [1]:
import pandas as pd
import numpy as np
import os
import time
import sys
import requests
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import json
from random import randint
from sqlalchemy import create_engine
from datetime import datetime, timedelta
from observations import get_observation
sys.path.append('../../core')
from mysql import MYSQL

In [13]:
pd.options.mode.chained_assignment = None
# db_connection = 'mysql+pymysql://douzi@traffic110:1qaz!QAZ2wsx@WSX@traffic110.mysql.database.azure.com/traffic'
db_connection = 'mysql+pymysql://root:@localhost/accident'
conn = create_engine(db_connection)
df = pd.read_sql("""
                 SELECT a.ACCIDENT_NO, ACCIDENTDATE, ACCIDENTTIME, n.Lat, n.Long, a.SPEED_ZONE  FROM ACCIDENT as a
                        LEFT JOIN NODE as n ON a.ACCIDENT_NO = n.ACCIDENT_NO
                        """, conn)

In [14]:
for p in range(df.shape[0]):
    if df['ACCIDENT_NO'][p] == 'T20180000340':
        kill = p
print(kill)
df = df.loc[kill:]
df.reset_index(level=None,drop=True, inplace=True)
print(df)

167460
       ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME       Lat       Long  SPEED_ZONE
0     T20180000340   2017-12-30     20:00:00 -37.93495  145.04957          70
1     T20180000341   2018-01-06     14:00:08 -37.76901  145.12282          50
2     T20180000343   2018-01-06     17:00:00 -37.81670  144.89103          50
3     T20180000351   2018-01-06     23:00:00 -38.10612  147.06399          50
4     T20180000352   2018-01-06     15:00:05 -37.06743  144.22230          60
5     T20180000356   2018-01-07     00:00:04 -34.23895  142.09712          80
6     T20180000359   2018-01-07     00:00:05 -37.63102  144.92666          60
7     T20180000360   2018-01-03     13:00:00 -38.00228  145.21775          60
8     T20180000365   2018-01-07     10:00:00 -37.86910  147.01919         100
9     T20180000366   2018-01-06     13:00:00 -37.51763  145.87339         100
10    T20180000367   2017-10-22     14:00:00 -37.79305  144.97572          60
11    T20180000372   2018-01-07     11:00:00 -37.80924  1

In [15]:
# ms = MYSQL(host="traffic110.mysql.database.azure.com",user="douzi@traffic110",pwd="1qaz!QAZ2wsx@WSX",db="traffic")
ms = MYSQL(host="localhost",user="root",pwd="",db="accident")

In [16]:
df['ACCIDENTDATE'] = pd.to_datetime(df['ACCIDENTDATE'])
df['ACCIDENTDATE_FOMAT'] = df['ACCIDENTDATE'].dt.strftime('%Y%m%d')
df['ACCIDENTTIME'] = pd.to_datetime(df['ACCIDENTTIME']).dt.strftime('%H:00:00')
df['Datetime'] = pd.to_datetime(df['ACCIDENTDATE'].apply(str) + ' ' + df['ACCIDENTTIME'])
df['Timestrap'] = df['Datetime'].values.astype(np.int64) // 10 ** 9
print(df)

       ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME       Lat       Long  SPEED_ZONE  \
0     T20180000340   2017-12-30     20:00:00 -37.93495  145.04957          70   
1     T20180000341   2018-01-06     14:00:00 -37.76901  145.12282          50   
2     T20180000343   2018-01-06     17:00:00 -37.81670  144.89103          50   
3     T20180000351   2018-01-06     23:00:00 -38.10612  147.06399          50   
4     T20180000352   2018-01-06     15:00:00 -37.06743  144.22230          60   
5     T20180000356   2018-01-07     00:00:00 -34.23895  142.09712          80   
6     T20180000359   2018-01-07     00:00:00 -37.63102  144.92666          60   
7     T20180000360   2018-01-03     13:00:00 -38.00228  145.21775          60   
8     T20180000365   2018-01-07     10:00:00 -37.86910  147.01919         100   
9     T20180000366   2018-01-06     13:00:00 -37.51763  145.87339         100   
10    T20180000367   2017-10-22     14:00:00 -37.79305  144.97572          60   
11    T20180000372   2018-01

In [453]:
#https://api.weather.com/v1/geocode/-37.688/144.841/observations/historical.json?apiKey=6532d6454b8aa370768e63d6ba5a832e&startDate=20060101&endDate=20060102&units=e
base_url = 'https://api.weather.com/v1/geocode/'
format_url = '/observations/historical.json?units=e'
api_key = '&apiKey=6532d6454b8aa370768e63d6ba5a832e'


In [454]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [470]:
print(df.head())

    ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME       Lat       Long  SPEED_ZONE  \
0  T20060000010   2006-01-13     12:00:00 -37.98862  145.21805          60   
1  T20060000018   2006-01-13     19:00:00 -37.99092  145.27632          70   
2  T20060000022   2006-01-14     12:00:00 -38.39632  144.85489         100   
3  T20060000023   2006-01-14     11:00:00 -37.98918  145.14496          80   
4  T20060000026   2006-01-14     10:00:00 -38.37299  144.87159          50   

  ACCIDENTDATE_FOMAT            Datetime   Timestrap  snowing  raining  foggy  \
0           20060113 2006-01-13 12:00:00  1137153600        0        0      0   
1           20060113 2006-01-13 19:00:00  1137178800        0        0      0   
2           20060114 2006-01-14 12:00:00  1137240000        0        0      0   
3           20060114 2006-01-14 11:00:00  1137236400        0        0      0   
4           20060114 2006-01-14 10:00:00  1137232800        0        0      0   

   smoke  dust  strong_winds wind_dir wind_s

In [474]:
df.reset_index(level=None,drop=True, inplace=True)
# for i in range(df.shape[0]):
df['snowing'] = 0
df['raining'] = 0
df['foggy'] = 0
df['smoke'] = 0
df['dust'] = 0
df['strong_winds'] = 0
df['wind_dir'] = ''
df['wind_speed'] = ''
df['temperature'] = 0

for i in range(3):
    url = base_url + str(df['Lat'][i]) + '/' + str(df['Long'][i]) + format_url + '&startDate=' + df['ACCIDENTDATE_FOMAT'][i] + '&endDate=' + df['ACCIDENTDATE_FOMAT'][i] + api_key
    print(url)
    print(df['ACCIDENT_NO'][i])
    print(df['Lat'][i], df['Long'][i])
    res = requests.get(url).json()
#     time.sleep(1)
    
    while res['metadata']['status_code'] != 200:
        url = base_url + '-37.' + str(randint(0, 9999)) + '/' + '144.' + str(randint(0, 999)) + format_url + '&startDate=' + df['ACCIDENTDATE_FOMAT'][i] + '&endDate=' + df['ACCIDENTDATE_FOMAT'][i] + api_key
        try:
            res = requests.get(url).json()
        except:
            pass
    
#     if r['metadata']['status_code'] != 200:
#         url = base_url + '-37.' + str(randint(0, 9999)) + '/' + '144.' + str(randint(0, 999)) + format_url + '&startDate=' + df['ACCIDENTDATE_FOMAT'][i] + '&endDate=' + df['ACCIDENTDATE_FOMAT'][i] + api_key
#         print(url)
#         res = requests.get(url).json()
# #         time.sleep(1)
#     else:
#         res = r
#     if res['metadata']['status_code'] == 400:
#         time.sleep(2)
#         res = requests.get(url).json()

    w_date = []
    print(res['metadata']['status_code'])
    for j in range(len(res['observations'])):
        ts = int(res['observations'][j]['valid_time_gmt'])
        w_time = datetime.utcfromtimestamp(ts)
        w_time = w_time + timedelta(hours=11)
        w_date.append(w_time)
    getDate = nearest(w_date, df['Datetime'][i])
    for k in range(len(w_date)):
        if getDate == w_date[k]:
            df['wind_dir'][i] = res['observations'][k]['wdir_cardinal']
            df['wind_speed'][i] = res['observations'][k]['wspd']
            df['temperature'][i] = res['observations'][k]['temp']
    
    getWeather = ms.ExecQuery("""
                   SELECT ATMOSPH_COND FROM ATMOSPHERIC_COND WHERE ACCIDENT_NO = '%s'
                   """ % (df['ACCIDENT_NO'][i]))
    get_weather = getWeather[0][0]
    if get_weather == 2:
        df['raining'][i] = 1
    elif get_weather == 3:
        df['snowing'][i] = 1
    elif get_weather == 4:
        df['foggy'][i] = 1
    elif get_weather == 5:
        df['smoke'][i] = 1
    elif get_weather == 6:
        df['dust'][i] = 1
    elif get_weather == 7:
        df['strong_winds'][i] = 1
    else:
        print('Sunny day')
    print(df.loc[[i]])
#     with open('my_csv.csv', 'a') as f:
#         f.write(df.iloc[i].values)
    df.iloc[[i]].to_csv('my_csv.csv', mode='a', header=False)
#     try:
#         ms.ExecNonQuery("""
#                     INSERT INTO accident_weather 
#                         (accident_no, wind_dir, wind_speed, temperature, snowing, raining, foggy, smoke, dust, strong_winds)
#                             VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')
#                     """ % (df['ACCIDENT_NO'][i], df['wind_dir'][i], df['wind_speed'][i], df['temperature'][i], df['snowing'][i], df['raining'][i], df['foggy'][i], df['smoke'][i], df['dust'][i], df['strong_winds'][i]))
#     except:
#         print('Add data error')
#     df.drop(index=i, inplace=True)
    

https://api.weather.com/v1/geocode/-37.98862/145.21805/observations/historical.json?units=e&startDate=20060113&endDate=20060113&apiKey=6532d6454b8aa370768e63d6ba5a832e
T20060000010
-37.98862 145.21805
200
Sunny day
    ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME       Lat       Long  SPEED_ZONE  \
0  T20060000010   2006-01-13     12:00:00 -37.98862  145.21805          60   

  ACCIDENTDATE_FOMAT            Datetime   Timestrap  snowing  raining  foggy  \
0           20060113 2006-01-13 12:00:00  1137153600        0        0      0   

   smoke  dust  strong_winds wind_dir wind_speed  temperature  
0      0     0             0      SSE         12           65  
https://api.weather.com/v1/geocode/-37.99092/145.27632/observations/historical.json?units=e&startDate=20060113&endDate=20060113&apiKey=6532d6454b8aa370768e63d6ba5a832e
T20060000018
-37.99092 145.27632
200
Sunny day
    ACCIDENT_NO ACCIDENTDATE ACCIDENTTIME       Lat       Long  SPEED_ZONE  \
1  T20060000018   2006-01-13     19:00:00 -3